In [2]:
from prefect import flow, task, variables, serve 
from prefect.filesystems import RemoteFileSystem
import pandas as pd
import requests
import datetime
import io
from typing import Optional

import sqlalchemy as db
import yfinance as yf

In [7]:
engine = db.create_engine(await variables.get('q_data_db_connect_url'))
conn = engine.connect()

In [ ]:
r = conn.execute(db.text("""
    SELECT symbol, exchange 
    FROM stock_meta
    LIMIT 10;
             """))

In [ ]:
tickers = yf.Ticker('aapl')

In [ ]:
tickers.history(start=(datetime.date.today() - datetime.timedelta(days=3.0)), end=(datetime.date.today() - datetime.timedelta(days=2.0)), interval='1m', prepost=True)

In [ ]:
today_str = datetime.date.today().strftime('%Y-%m-%d')
next_str = (datetime.date.today() - datetime.timedelta(days=2.0)).strftime('%Y-%m-%d')
next_str_1 = (datetime.date.today() - datetime.timedelta(days=1.0)).strftime('%Y-%m-%d')

In [ ]:
info = tickers.info

In [ ]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')

In [ ]:
nyse = mcal.get_calendar('NASDAQ')

In [ ]:
nyse.valid_days(datetime.date.today() - datetime.timedelta(days=2.0), datetime.date.today() - datetime.timedelta(days=1.0)).size

In [ ]:
datetime.date.today().weekday()

In [ ]:
pd.DataFrame.from_dict(info, orient='index')

In [ ]:
del info['companyOfficers']

In [ ]:
info_df = pd.DataFrame(info, index=[0])

In [ ]:
tickers.options

In [ ]:
opt = tickers.option_chain(date='2023-12-22')

In [ ]:
opt.calls

In [ ]:
opt.puts

In [ ]:
r = conn.execute(db.text("""
    SELECT symbol, exchange
    FROM stock_meta
    LIMIT 10;
    """))

In [ ]:
result = r.fetchall()

In [ ]:
s, e = result[1]

In [ ]:
e

In [5]:
fs = await RemoteFileSystem.load("yfinance-daily")

In [ ]:
await fs.write_path('test', b'')

In [8]:
r = conn.execute(db.text("""
   SELECT DISTINCT file_path, symbol, data_date FROM yf_bad_record
   WHERE symbol='A' AND file_type='1m-stock' AND bucket_name='yfinance-daily' AND error_code=0
    ORDER BY data_date;
    """))
result = r.fetchall()
result

In [18]:
df = pd.read_csv(io.BytesIO(x), header=0)

In [19]:
df.Datetime = pd.to_datetime(df.Datetime)

In [158]:
print(pd.io.sql.get_schema(df, name='A'))

CREATE TABLE "A" (
"Datetime" TIMESTAMP,
  "Open" REAL,
  "High" REAL,
  "Low" REAL,
  "Close" REAL,
  "Volume" INTEGER,
  "Dividends" REAL,
  "Stock Splits" REAL
)


In [28]:
create_table = """
CREATE TABLE IF NOT EXISTS A (
  "Datetime" TIMESTAMP PRIMARY KEY,
  "Open" REAL,
  "High" REAL,
  "Low" REAL,
  "Close" REAL,
  "Volume" INTEGER,
  "Dividends" REAL,
  "Stock Splits" REAL
"""

make_it_timescale = """
    SELECT create_hypertable('A', by_range('Datetime'));
"""

In [12]:
db.inspect(engine).has_table('yf_bad_record')

True

In [41]:
def create_time_scale_table(table_name, conn: db.Connection):
    create_table = db.text(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            datetime TIMESTAMPTZ NOT NULL PRIMARY KEY,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume INTEGER,
            dividends REAL,
            stock_splits REAL);
            """)

    make_it_timescale = db.text(f"""
        SELECT create_hypertable('{table_name}', by_range('datetime'));
        """)

    conn.execute(create_table)
    conn.execute(make_it_timescale)


def inset_row(df: pd.DataFrame, table_name: str):
    for index, item in df.iterrows(): 
        insert_stmt = db.text(f"""
            INSERT INTO {table_name} (
                "datetime",
                "open",
                "high",
                "low",
                "close",
                "volume",
                "dividends",
                "stock_splits") VALUES (
                '{item.Datetime}',
                '{item.Open}',
                '{item.High}',
                '{item.Low}',
                '{item.Close}',
                '{item.Volume}',
                '{item.Dividends}',
                '{item["Stock Splits"]}'
            ) ON CONFLICT DO NOTHING
        """)
        conn.execute(insert_stmt)

conn = engine.connect()
if not db.inspect(engine).has_table('a'):
    create_time_scale_table('a', conn)

inset_row(df, 'a')
conn.commit()
conn.close()

In [171]:
df['Stock Splits']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
406    0.0
407    0.0
408    0.0
409    0.0
410    0.0
Name: Stock Splits, Length: 411, dtype: float64

In [24]:
df

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-12-11 06:28:00-05:00,127.21,127.21,127.21,127.21,0,0.0,0.0
1,2023-12-11 06:42:00-05:00,127.21,127.21,127.21,127.21,0,0.0,0.0
2,2023-12-11 07:00:00-05:00,127.20,127.20,125.50,127.00,0,0.0,0.0
3,2023-12-11 07:15:00-05:00,127.00,127.00,127.00,127.00,0,0.0,0.0
4,2023-12-11 07:35:00-05:00,127.00,127.00,127.00,127.00,0,0.0,0.0
...,...,...,...,...,...,...,...,...
406,2023-12-11 16:37:00-05:00,128.97,128.97,128.97,128.97,0,0.0,0.0
407,2023-12-11 16:52:00-05:00,128.97,128.97,128.97,128.97,0,0.0,0.0
408,2023-12-11 17:07:00-05:00,128.97,128.97,128.97,128.97,0,0.0,0.0
409,2023-12-11 17:16:00-05:00,128.97,128.97,128.97,128.97,0,0.0,0.0
